In [1]:
%matplotlib inline
from __future__ import print_function
from PIL import Image
import numpy as np
import tensorflow as tf
import os
import glob
import matplotlib.pyplot as plt

/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def conv_block(inputs, out_channels, name='conv'):
    with tf.variable_scope(name):
        conv = tf.layers.conv2d(inputs, out_channels, kernel_size=3, padding='SAME')
        conv = tf.contrib.layers.batch_norm(conv, updates_collections=None, decay=0.99, scale=True, center=True)
        conv = tf.nn.relu(conv)
        conv = tf.contrib.layers.max_pool2d(conv, 2)
        return conv

In [3]:
def encoder(x, h_dim, z_dim, reuse=False):
    with tf.variable_scope('encoder', reuse=reuse):
        net = conv_block(x, h_dim, name='conv_1')
        net = conv_block(net, h_dim, name='conv_2')
        net = conv_block(net, h_dim, name='conv_3')
        net = conv_block(net, z_dim, name='conv_4')
        net = tf.contrib.layers.flatten(net)
        return net

In [4]:
def euclidean_distance(a, b):
    # a.shape = N x D
    # b.shape = M x D
    N, D = tf.shape(a)[0], tf.shape(a)[1]
    M = tf.shape(b)[0]
    a = tf.tile(tf.expand_dims(a, axis=1), (1, M, 1))
    b = tf.tile(tf.expand_dims(b, axis=0), (N, 1, 1))
    return tf.reduce_mean(tf.square(a - b), axis=2)

In [5]:
n_epochs = 100
n_episodes = 100
n_way = 20
n_shot = 5
n_query = 15
n_examples = 350
im_width, im_height, channels = 84, 84, 3
h_dim = 64
z_dim = 64

In [6]:
# Load Train Dataset
train_dataset = np.load('mini-imagenet-train.npy')
n_classes = train_dataset.shape[0]
print(train_dataset.shape)

(64, 350, 84, 84, 3)


In [7]:
x = tf.placeholder(tf.float32, [None, None, im_height, im_width, channels])
q = tf.placeholder(tf.float32, [None, None, im_height, im_width, channels])
x_shape = tf.shape(x)
q_shape = tf.shape(q)
num_classes, num_support = x_shape[0], x_shape[1]
num_queries = q_shape[1]
y = tf.placeholder(tf.int64, [None, None])
y_one_hot = tf.one_hot(y, depth=num_classes)
emb_in = encoder(tf.reshape(x, [num_classes * num_support, im_height, im_width, channels]), h_dim, z_dim)
emb_dim = tf.shape(emb_in)[-1]
emb_x = tf.reduce_mean(tf.reshape(emb_in, [num_classes, num_support, emb_dim]), axis=1)
emb_q = encoder(tf.reshape(q, [num_classes * num_queries, im_height, im_width, channels]), h_dim, z_dim, reuse=True)
dists = euclidean_distance(emb_q, emb_x)
log_p_y = tf.reshape(tf.nn.log_softmax(-dists), [num_classes, num_queries, -1])
ce_loss = -tf.reduce_mean(tf.reshape(tf.reduce_sum(tf.multiply(y_one_hot, log_p_y), axis=-1), [-1]))
acc = tf.reduce_mean(tf.to_float(tf.equal(tf.argmax(log_p_y, axis=-1), y)))

In [8]:
train_op = tf.train.AdamOptimizer().minimize(ce_loss)

In [9]:
sess = tf.InteractiveSession()
init_op = tf.global_variables_initializer()
sess.run(init_op)

In [10]:
for ep in range(n_epochs):
    for epi in range(n_episodes):
        epi_classes = np.random.permutation(n_classes)[:n_way]
        support = np.zeros([n_way, n_shot, im_height, im_width, channels], dtype=np.float32)
        query = np.zeros([n_way, n_query, im_height, im_width, channels], dtype=np.float32)
        for i, epi_cls in enumerate(epi_classes):
            selected = np.random.permutation(n_examples)[:n_shot + n_query]
            support[i] = train_dataset[epi_cls, selected[:n_shot]]
            query[i] = train_dataset[epi_cls, selected[n_shot:]]
        # support = np.expand_dims(support, axis=-1)
        # query = np.expand_dims(query, axis=-1)
        labels = np.tile(np.arange(n_way)[:, np.newaxis], (1, n_query)).astype(np.uint8)
        _, ls, ac = sess.run([train_op, ce_loss, acc], feed_dict={x: support, q: query, y:labels})
        if (epi+1) % 50 == 0:
            print('[epoch {}/{}, episode {}/{}] => loss: {:.5f}, acc: {:.5f}'.format(ep+1, n_epochs, epi+1, n_episodes, ls, ac))

[epoch 1/100, episode 50/100] => loss: 2.69017, acc: 0.20667
[epoch 1/100, episode 100/100] => loss: 2.62193, acc: 0.22000
[epoch 2/100, episode 50/100] => loss: 2.51031, acc: 0.31000
[epoch 2/100, episode 100/100] => loss: 2.45286, acc: 0.23667
[epoch 3/100, episode 50/100] => loss: 2.42127, acc: 0.28000
[epoch 3/100, episode 100/100] => loss: 2.26645, acc: 0.30000
[epoch 4/100, episode 50/100] => loss: 2.38223, acc: 0.28667
[epoch 4/100, episode 100/100] => loss: 2.24251, acc: 0.33000
[epoch 5/100, episode 50/100] => loss: 2.38313, acc: 0.27333
[epoch 5/100, episode 100/100] => loss: 2.31108, acc: 0.29000
[epoch 6/100, episode 50/100] => loss: 2.47622, acc: 0.24667
[epoch 6/100, episode 100/100] => loss: 2.10669, acc: 0.35667
[epoch 7/100, episode 50/100] => loss: 2.18633, acc: 0.32000
[epoch 7/100, episode 100/100] => loss: 1.98386, acc: 0.40333
[epoch 8/100, episode 50/100] => loss: 2.14229, acc: 0.33000
[epoch 8/100, episode 100/100] => loss: 2.06077, acc: 0.34000
[epoch 9/100, ep

[epoch 67/100, episode 50/100] => loss: 0.88438, acc: 0.76333
[epoch 67/100, episode 100/100] => loss: 0.86021, acc: 0.70000
[epoch 68/100, episode 50/100] => loss: 0.90512, acc: 0.71333
[epoch 68/100, episode 100/100] => loss: 0.79389, acc: 0.77000
[epoch 69/100, episode 50/100] => loss: 0.91446, acc: 0.72667
[epoch 69/100, episode 100/100] => loss: 0.89832, acc: 0.69333
[epoch 70/100, episode 50/100] => loss: 0.95540, acc: 0.67333
[epoch 70/100, episode 100/100] => loss: 1.07609, acc: 0.62667
[epoch 71/100, episode 50/100] => loss: 1.08928, acc: 0.64333
[epoch 71/100, episode 100/100] => loss: 0.90997, acc: 0.67000
[epoch 72/100, episode 50/100] => loss: 0.79448, acc: 0.72333
[epoch 72/100, episode 100/100] => loss: 0.77371, acc: 0.73667
[epoch 73/100, episode 50/100] => loss: 0.63826, acc: 0.79000
[epoch 73/100, episode 100/100] => loss: 0.75410, acc: 0.76000
[epoch 74/100, episode 50/100] => loss: 0.96961, acc: 0.64667
[epoch 74/100, episode 100/100] => loss: 0.88101, acc: 0.67667


In [11]:
# Load Test Dataset
test_dataset = np.load('mini-imagenet-test.npy')
n_test_classes = test_dataset.shape[0]
print(test_dataset.shape)

(20, 350, 84, 84, 3)


In [12]:
n_test_episodes = 600
n_test_way = 5
n_test_shot = 5
n_test_query = 15

In [13]:
print('Testing...')
avg_acc = 0.
for epi in range(n_test_episodes):
    epi_classes = np.random.permutation(n_test_classes)[:n_test_way]
    support = np.zeros([n_test_way, n_test_shot, im_height, im_width, channels], dtype=np.float32)
    query = np.zeros([n_test_way, n_test_query, im_height, im_width, channels], dtype=np.float32)
    for i, epi_cls in enumerate(epi_classes):
        selected = np.random.permutation(n_examples)[:n_test_shot + n_test_query]
        support[i] = test_dataset[epi_cls, selected[:n_test_shot]]
        query[i] = test_dataset[epi_cls, selected[n_test_shot:]]
    # support = np.expand_dims(support, axis=-1)
    # query = np.expand_dims(query, axis=-1)
    labels = np.tile(np.arange(n_test_way)[:, np.newaxis], (1, n_test_query)).astype(np.uint8)
    ls, ac = sess.run([ce_loss, acc], feed_dict={x: support, q: query, y:labels})
    avg_acc += ac
    if (epi+1) % 50 == 0:
        print('[test episode {}/{}] => loss: {:.5f}, acc: {:.5f}'.format(epi+1, n_test_episodes, ls, ac))
avg_acc /= n_test_episodes
print('Average Test Accuracy: {:.5f}'.format(avg_acc))

Testing...
[test episode 50/600] => loss: 1.05772, acc: 0.58667
[test episode 100/600] => loss: 1.33291, acc: 0.48000
[test episode 150/600] => loss: 0.58527, acc: 0.73333
[test episode 200/600] => loss: 0.65924, acc: 0.69333
[test episode 250/600] => loss: 1.12209, acc: 0.52000
[test episode 300/600] => loss: 0.97975, acc: 0.68000
[test episode 350/600] => loss: 1.05668, acc: 0.65333
[test episode 400/600] => loss: 0.98864, acc: 0.68000
[test episode 450/600] => loss: 1.04565, acc: 0.65333
[test episode 500/600] => loss: 0.72200, acc: 0.72000
[test episode 550/600] => loss: 0.77793, acc: 0.77333
[test episode 600/600] => loss: 0.94674, acc: 0.62667
Average Test Accuracy: 0.61807
